# Process and prepare Met Office weather data for use in CityLearn environment

The weather variables required are: "Outdoor Drybulb Temperature [C], Relative Humidity [%], Diffuse Solar Radiation [W/m2], Direct Solar Radiation [W/m2]"

I have decided to use:
- the `bedford` station for temperatue and humidity, from this dataset https://catalogue.ceda.ac.uk/uuid/c9663d0c525f4b0698f1ec4beae3688e (cambridge-botanic-garden only has daily data, and cambridge-niab has terrible data availability)

I will also process data from:
- the `bedford` station for solar irradiation data, from this dataset https://catalogue.ceda.ac.uk/uuid/87eb67c08f5c4518a3723d0ca2d9b4b1 (unfortunately this station only has global irradiation data, and no direct-diffuse breakdown - no stations near Cambridge have this)

for both `qc-version-1` data is used.

An alternative weather dataset is available from Meteostat, https://meteostat.net/en/station/EGSC0, however this data has worse availability, and so the MIDAS dataset is used by preference.

## Notes on solar irradiance

The Met Office MIDAS dataset only provides global irradiance values.

However, renewables.ninja provides direct and diffuse components of solar irradiance which are used in its calculation of pv power generation. But, these values are dervied from MERRA-2 reanalysis data, as described in, https://www.sciencedirect.com/science/article/pii/S0360544216311744, and that reference states "The global coverage of reanalysis data may come at the cost of accuracy [4]. examine two reanalyses (ERA-Interim and MERRA) and show that their irradiance values are less accurate than satellite-derived data, frequently predicting clear skies when the sky was cloudy."

But, as these are the values that are used in the calculation of the solar generation power, they will likely have greater correlation to the power values (which are the variables of importance to the control problem), and so will provide better predictors.

However, the irradiance values provided by renewables.ninja do not have a unit explicitly associated. Reading between the lines fo the documentation, they appear to be in [kW/m2] - see solar data processing ipynb for more detail.

An alternative option would be to use the renewables.ninja data to get an estimate of the diffuse-to-direct irradiance ratio, and then combine that with the real measurement Met Office data to estimate the diffuse and direct components.

In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
years = list(range(2000,2023))
in_dir = 'raw_data'

## Prep weather data (temperature & relative humidity)

In the Met MIDAS dataset:
- 'Outdoor Drybulb Temperature [C]' is given by variable `air_temperature`
- 'Relative Humidity [%]' is given by variable `rltv_hum`

In [3]:
# do I want to get this automatically from the metadata csv file?
weather_metadata = {
    'station name': 'bedford',
    'location': [52.227, -0.465],
    'MetOff station id': 3560
}

In [4]:
subdir = 'hourly_weather'
in_fname_pattern = 'midas-open_uk-hourly-weather-obs_dv-202308_bedfordshire_00461_bedford_qcv-1_%s.csv'
in_fpath_pattern = os.path.join(in_dir,subdir,in_fname_pattern)

In [5]:
weather_observations = pd.DataFrame()

for year in years:
    year_data = pd.read_csv(
        in_fpath_pattern%year,
        usecols=['ob_time','air_temperature','rltv_hum'],
        skiprows=range(280),
        skipfooter=1,
        engine='python'
        )
    weather_observations = pd.concat([weather_observations,year_data],ignore_index=True)

In [6]:
print(weather_observations)

                    ob_time  air_temperature  rltv_hum
0       2000-01-01 00:00:00              8.1      94.3
1       2000-01-01 01:00:00              8.2      95.8
2       2000-01-01 02:00:00              8.1      97.2
3       2000-01-01 03:00:00              8.2      97.2
4       2000-01-01 04:00:00              8.0      97.2
...                     ...              ...       ...
199024  2022-12-31 19:00:00             11.3      94.3
199025  2022-12-31 20:00:00             11.4      92.4
199026  2022-12-31 21:00:00             11.6      88.1
199027  2022-12-31 22:00:00             11.8      82.6
199028  2022-12-31 23:00:00             11.2      82.9

[199029 rows x 3 columns]


In [7]:
# split into dataframes for each variable
temp_df = weather_observations[['ob_time','air_temperature']].copy()
hum_df = weather_observations[['ob_time','rltv_hum']].copy()

## Prep solar data (global irradiation only)

The only relevant variable available is `glbl_irad_amt`, which is the "Global solar irradiation amount" in [KJ/m2]

According to the table in Section 4.1.4 of the documentation (https://zenodo.org/record/7357325), the "Global radiation" is "Total global radiation over preceding hour" in "W hr/m2" (this isn't right, it is in kJ/m2).
So to produce the required [W/m2] value, the data needs to be mulitplied by $1000[J/kJ]/3600[s/hr]$

In [8]:
# do I want to get this automatically from the metadata csv file?
solar_metadata = {
    'station name': 'bedford',
    'location': [52.227, -0.465],
    'MetOff station id': 3440
}

In [9]:
subdir = 'hourly_solar'
fname_pattern = 'midas-open_uk-radiation-obs_dv-202308_bedfordshire_00461_bedford_qcv-1_%s.csv'
fpath_pattern = os.path.join(in_dir,subdir,fname_pattern)

In [10]:
irad_df = pd.DataFrame()

for year in years:
    year_data = pd.read_csv(
        fpath_pattern%year,
        usecols=['ob_end_time','id_type','glbl_irad_amt'],
        skiprows=range(75),
        skipfooter=1,
        engine='python'
        )
    irad_df = pd.concat([irad_df,year_data],ignore_index=True)

irad_df = irad_df[irad_df['id_type'] == 'DCNN']
irad_df.drop('id_type',axis=1,inplace=True) # delete type col after filtering

# convert to units of W/m2
irad_df['glbl_irad_amt']  = np.around(irad_df['glbl_irad_amt']*(1000/3600), 1)

# rename timestamps column for consistency with weather data
irad_df.rename(columns={'ob_end_time':'ob_time'},inplace=True)

In [11]:
print(irad_df)

                    ob_time  glbl_irad_amt
0       2000-01-01 00:00:00            0.0
1       2000-01-01 01:00:00            0.0
2       2000-01-01 02:00:00            0.0
3       2000-01-01 03:00:00            0.0
4       2000-01-01 04:00:00            0.0
...                     ...            ...
198501  2022-12-31 19:00:00            0.0
198502  2022-12-31 20:00:00            0.0
198503  2022-12-31 21:00:00            0.0
198504  2022-12-31 22:00:00            0.0
198505  2022-12-31 23:00:00            0.0

[198044 rows x 2 columns]


## Check & clean data

In [12]:
data_dfs = [temp_df, hum_df, irad_df]
vars = ['air_temperature','rltv_hum','glbl_irad_amt']

In [13]:
def clean_data_df(df, var):
    """Clean dataframe containing observations for a single data variable.
    Strip out NaNs and fill observations gaps using linear interpolation,
    of two kinds depending on the duration of the gap.

    Args:
        df (pandas.Dataframe): Dataframe containing observation data for the variable
        var (str): name of the variable column in the dataframe

    Returns:
        clean_df (pandas.Dataframe): dataframe of cleaned data
    """
    # identify unavailable hours (either missing or NaN values)
    df['ob_time'] = pd.to_datetime(df['ob_time'])
    available_hours = list(df['ob_time'])
    missing_hours = pd.date_range(start=available_hours[0], end=available_hours[-1], freq='h').difference(available_hours)
    nan_hours = df[np.isnan(df[var])]['ob_time']

    unav_hours = missing_hours.append(pd.DatetimeIndex(nan_hours))
    unav_hours = unav_hours.sort_values()

    unav_ranges = []
    unav_range_durations = []
    if len(unav_hours) > 0:
        consec_unav = unav_hours[1:]-unav_hours[:-1]  == pd.Timedelta(hours=1)
        start = unav_hours[0]
        for t in range(len(unav_hours)-1):
            if not consec_unav[t]:
                unav_ranges.append([start,unav_hours[t]])
                unav_range_durations.append(unav_hours[t]-start)
                start = unav_hours[t+1]
        unav_ranges.append([start,unav_hours[-1]])
        unav_range_durations.append(unav_hours[-1]-start)
    else:
        print("No unavailable hours!")

    # remove times with NaN observations
    clean_df = df[np.invert(np.isnan(df[var]))]
    clean_df.reset_index(inplace=True, drop=True)

    # clean data (fill in unavailable values)
    insert_df = pd.DataFrame()

    for (window, duration) in list(zip(unav_ranges, unav_range_durations)):

        # for short durations use simple linear interpolation between adjacent times
        if duration <= pd.Timedelta(4, 'hours'):
            before_time = pd.Timestamp(window[0]) + pd.Timedelta(-1, 'hours')
            before_time_index = clean_df.index[clean_df['ob_time']==before_time].to_list()[0]
            after_time = pd.Timestamp(window[1]) + pd.Timedelta(1, 'hours')
            after_time_index = clean_df.index[clean_df['ob_time']==after_time].to_list()[0]
            gap_length = (after_time - before_time).total_seconds()

            for tstamp in pd.date_range(start=window[0], end=window[1], freq='h'):
                insert_obvs = {'ob_time':[tstamp]}
                dx = (tstamp - before_time).total_seconds()
                dy = (clean_df.loc[after_time_index][var] - clean_df.loc[before_time_index][var]) * dx/gap_length
                insert_obvs[var] = clean_df.loc[before_time_index][var] + dy
                insert_df = pd.concat([insert_df, pd.DataFrame(insert_obvs)])

    clean_df = pd.concat([clean_df,insert_df], ignore_index=True)
    clean_df.sort_values('ob_time', inplace=True, ignore_index=True)

    insert_df = pd.DataFrame()

    for (window, duration) in list(zip(unav_ranges, unav_range_durations)):

        # for longer durations use linear interpolation between observed values for
        # the same hour on the days with available data on either side.
        if duration > pd.Timedelta(4, 'hours'):
            # NOTE: this method only works when the data is hourly on the hour - can I think of a smarter,
            # more general way to do this??
            # Also, we assume that long duration unavailable windows are not adjacent, otherwise this breaks
            for tstamp in pd.date_range(start=window[0], end=window[1], freq='h'):
                insert_obvs = {'ob_time':[tstamp]}

                h = tstamp.hour
                # compute number of hours needed to go back from the window start to find timestamp with same hour on day before unavailable window
                hours_back = pd.Timestamp(window[0]).hour - h if h < pd.Timestamp(window[0]).hour else 24 - (h - pd.Timestamp(window[0]).hour)
                before_time = pd.Timestamp(window[0]) - pd.Timedelta(hours_back, 'hours')
                # check if this time is available, if not step one day before until value available
                while len(clean_df.index[clean_df['ob_time']==before_time].to_list()) == 0:
                    before_time = before_time + pd.Timedelta(-1, 'days')
                before_time_index = clean_df.index[clean_df['ob_time']==before_time].to_list()[0]

                # compute number of hours needed to go forwards from the window end to find timestamp with same hour on day after unavailable window
                hours_forward = h - pd.Timestamp(window[1]).hour if h > pd.Timestamp(window[1]).hour else 24 - (pd.Timestamp(window[1]).hour - h)
                after_time = pd.Timestamp(window[1]) + pd.Timedelta(hours_forward, 'hours')
                while len(clean_df.index[clean_df['ob_time']==after_time].to_list()) == 0:
                    after_time = after_time + pd.Timedelta(1, 'days')
                after_time_index = clean_df.index[clean_df['ob_time']==after_time].to_list()[0]

                gap_length = (after_time - before_time).total_seconds()

                dx = (tstamp - before_time).total_seconds()
                dy = (clean_df.loc[after_time_index][var] - clean_df.loc[before_time_index][var]) * dx/gap_length
                insert_obvs[var] = clean_df.loc[before_time_index][var] + dy
                insert_df = pd.concat([insert_df, pd.DataFrame(insert_obvs)])

    clean_df = pd.concat([clean_df,insert_df], ignore_index=True)
    clean_df.sort_values('ob_time', inplace=True, ignore_index=True)

    # recheck for unavailable hours
    available_hours = list(clean_df['ob_time'])
    missing_hours = pd.date_range(start=available_hours[0], end=available_hours[-1], freq='h').difference(available_hours)
    nan_hours = clean_df[np.isnan(clean_df[var])]['ob_time']
    assert len(missing_hours) == len(nan_hours) == 0

    return clean_df

In [14]:
[clean_temp, clean_hum, clean_irad] = [clean_data_df(df, var) for df, var in zip(data_dfs, vars)]

## Combine observations and output

In [15]:
assert np.array_equal(clean_temp['ob_time'], clean_hum['ob_time'])
assert np.array_equal(clean_temp['ob_time'], clean_irad['ob_time'])
# this requires the first and last day of the desired year range to have
# available data for all variables

In [16]:
# combine cleaned data into a single dataframe
metoff_data = clean_temp.copy()
metoff_data['rltv_hum'] = clean_hum['rltv_hum']
metoff_data['glbl_irad_amt'] = clean_irad['glbl_irad_amt']

In [17]:
# renamed columns to provide variables units for clarity
metoff_data.rename(columns={
    'ob_time':'datetime',
    'air_temperature':'air_temperature [degC]',
    'rltv_hum':'rltv_hum [%]',
    'glbl_irad_amt':'glbl_irad_amt [W/m2]'
    },inplace=True)

In [18]:
metoff_data = metoff_data.round({'air_temperature [degC]':1, 'rltv_hum [%]':1, 'glbl_irad_amt [W/m2]':1})

In [19]:
# save processed data to csv by year
location = 'bedford' # - get from metadata?
save_fname = 'MetOff_weather-{0}_mdata.json'.format(location)
save_dir = os.path.join('processed_data',location)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for year in years:
    year_data = metoff_data[metoff_data['datetime'].dt.year == year].copy()
    year_data['datetime'] = year_data['datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    year_data.to_csv(os.path.join(save_dir,f'{year}.csv'), index=False)

# save metadata
mdata_save_path = os.path.join(save_dir,save_fname)
mdata_dict = {
    'location':location,
    'data_path':save_dir,
    'years':years,
    'weather_metdata':weather_metadata,
    'solar_metadata':solar_metadata
    }
with open(mdata_save_path, 'w') as file:
    json.dump(mdata_dict, file, indent=4)